# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096976


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/33 [00:00<?, ?it/s]

Rendering models:   6%|▌         | 2/33 [00:05<01:21,  2.64s/it]

Rendering models:  12%|█▏        | 4/33 [00:06<01:00,  2.08s/it]

Rendering models:  18%|█▊        | 6/33 [00:06<00:39,  1.47s/it]

Rendering models:  24%|██▍       | 8/33 [00:07<00:26,  1.05s/it]

...

Rendering models:  79%|███████▉  | 26/33 [00:08<00:00,  7.78it/s]

Rendering models:  88%|████████▊ | 29/33 [00:08<00:00,  9.16it/s]

Rendering models:  94%|█████████▍| 31/33 [00:08<00:00,  9.89it/s]

Rendering models: 100%|██████████| 33/33 [00:08<00:00, 11.34it/s]

not-logged-in-3b0b929f4475fd276561    0.004114
not-logged-in-52d766b03306c0c1f279    0.001449
Alexander00                           0.000599
not-logged-in-561399b67c6761e990ff    0.000990
not-logged-in-8cf2e6929490167ca6ab    0.001751
jnarayanbvg                           0.002996
awright5                              0.004503
not-logged-in-37432d74053c6a167ecb    0.074689
not-logged-in-c6917d9b19f41e6c2995    0.003748
nasseem                               0.192076
not-logged-in-4447ff92852aa7c13340    0.000580
theetick                              0.003848
casbee                                0.003541
Roksolana                             0.000296
sn346808                              0.047933
sn346808                              0.050824
sn346808                              0.039708
sn346808                              0.014904
not-logged-in-116eef09a8f6e73ef4da    0.003439
KrisKeogh                             0.013169
Sylverone                             0.000586
MaximeRoyer19

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())